In [7]:
from bs4 import BeautifulSoup
import requests
import json
import re
import pandas as pd
import time
import sqlite3

In [3]:
head = {'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
       'Accept-Encoding': 'gzip, deflate, br',
       'Referer':'https://www.google.com/',
       'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
bov = requests.get('https://sports.bovada.lv/golf', headers = head)

In [4]:
bov_beaut = BeautifulSoup(bov.content, 'lxml')
odds = bov_beaut.find('script', 'content-data')

bs4.element.Tag

In [17]:
reg = re.compile('\*\/.*\/\*')
results = reg.search(odds.text)
just_json = re.sub('\*\/|\/\*', '', results.group(0))
parsed_json = json.loads(just_json)

# Grab Meta-Data
event = parsed_json[0]['card_data']['items'][0]['itemList']['items'][0]['description']
description = parsed_json[0]['card_data']['items'][0]['itemList']['items'][0]['displayGroups'][0]['description']
market_type = parsed_json[0]['card_data']['items'][0]['itemList']['items'][0]['displayGroups'][0]['itemList'][0]['marketTypeGroup']
period_type = parsed_json[0]['card_data']['items'][0]['itemList']['items'][0]['displayGroups'][0]['itemList'][0]['periodType']
status = parsed_json[0]['card_data']['items'][0]['itemList']['items'][0]['displayGroups'][0]['itemList'][0]['status']
golfers_data = parsed_json[0]['card_data']['items'][0]['itemList']['items'][0]['displayGroups'][0]['itemList'][0]['outcomes']

# Collect Player Data
player = []
bet_id = []
american_odds = []
decimal_odds = []
for golfer in golfers_data:
    player.append(golfer['description'])
    bet_id.append(golfer['id'])
    american_odds.append(golfer['price']['american'])
    decimal_odds.append(golfer['price']['decimal'])

# Throw data into a pandas DataFrame
golf_df = pd.DataFrame({'player' : player, 
                        'bet_id' : bet_id,
                        'american_odds' : american_odds, 
                        'decimal_odds' : decimal_odds})

golf_df = golf_df[['player', 'bet_id', 'american_odds', 'decimal_odds']]
golf_df['event'], golf_df['description'], golf_df['market_type'], golf_df['period_type'], golf_df['status'], golf_df['date'] = [event, description, market_type, period_type, status, time.strftime("%m/%d/%Y")]
golf_df.head()

,player,bet_id,american_odds,decimal_odds,event,description,market_type,period_type,status,date
0,Dustin Johnson,165714937,+1200,13.00,British Open 2017,Main Markets,RANK_OUTRIGHT,Tournament,OPEN,07/16/2017
1,Jordan Spieth,165714950,+1200,13.00,British Open 2017,Main Markets,RANK_OUTRIGHT,Tournament,OPEN,07/16/2017
2,Rory McIlroy,165714965,+1400,15.00,British Open 2017,Main Markets,RANK_OUTRIGHT,Tournament,OPEN,07/16/2017
3,Rickie Fowler,165714964,+1400,15.00,British Open 2017,Main Markets,RANK_OUTRIGHT,Tournament,OPEN,07/16/2017
4,Jon Rahm,165714948,+1400,15.00,British Open 2017,Main Markets,RANK_OUTRIGHT,Tournament,OPEN,07/16/2017
